In [1]:
import einops
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras import models as M
import pandas as pd
import pathlib
import numpy as np
from tensorflow_text import normalize_utf8

2023-01-05 23:22:06.675253: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-05 23:22:06.870153: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-05 23:22:06.870192: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-05 23:22:07.997937: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Load dataset

In [2]:
path_to_file = pathlib.Path().home() / "tensorflow_datasets" / "anki" / "ita-eng" / "ita.txt"
df = pd.read_csv(path_to_file, sep="\t", header=None)
df.columns=["eng", "ita", "author"]
df["split"] = np.where(np.random.uniform(size=(len(df),)) < 0.8, "train", "val")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358373 entries, 0 to 358372
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   eng     358373 non-null  object
 1   ita     358373 non-null  object
 2   author  358373 non-null  object
 3   split   358373 non-null  object
dtypes: object(4)
memory usage: 10.9+ MB


In [3]:
df.head()

,eng,ita,author,split
0,Hi.,Ciao!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...,train
1,Hi.,Ciao.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...,val
2,Run!,Corri!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...,train
3,Run!,Corra!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...,val
4,Run!,Correte!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...,train


## Prep dataset

In [256]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 30000

In [5]:
train_raw = (
    tf.data.Dataset
    .from_tensor_slices((
        df.query("split == 'train'")["ita"].values,
        df.query("split == 'train'")["eng"].values,
    ))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

val_raw = (
    tf.data.Dataset
    .from_tensor_slices((
        df.query("split == 'val'")["ita"].values,
        df.query("split == 'val'")["eng"].values,
    ))
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
)

2023-01-05 23:22:10.602214: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-05 23:22:10.602241: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-05 23:22:10.602264: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (calcifer-Inspiron-7370): /proc/driver/nvidia/version does not exist
2023-01-05 23:22:10.602564: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
@tf.function
def clean_string(s):
    # Split accented characters
    text = normalize_utf8(s, "NFKD")
    text = tf.strings.lower(text)
    # Keep space, a to z, and select punctuation.
    text = tf.strings.regex_replace(text, '[^ a-z.?!,;]', '')
    # Add spaces around punctuation.
    text = tf.strings.regex_replace(text, '[.?!,;]', r' \0 ')
    # Strip whitespace.
    text = tf.strings.strip(text)
    # Add START and END tokens
    text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
    return text

In [7]:
vectorizer_src = L.TextVectorization(
    max_tokens=VOCAB_SIZE,
    standardize=clean_string,
    ragged=True,
    encoding='utf-8',
)
vectorizer_src.adapt(train_raw.map(lambda src, dst: src))

vectorizer_dst = L.TextVectorization(
    max_tokens=VOCAB_SIZE,
    standardize=clean_string,
    ragged=True,
    encoding='utf-8',
)

vectorizer_dst.adapt(train_raw.map(lambda src, dst: dst))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [8]:
def prep_dataset(src, dst):
    src_enc = vectorizer_src(src).to_tensor()
    dst_enc = vectorizer_dst(dst)
    dst_in = dst_enc[:, :-1].to_tensor()
    dst_out = dst_enc[:, 1:].to_tensor()
    return (src_enc, dst_in), dst_out

In [9]:
ds_train = train_raw.map(prep_dataset, tf.data.AUTOTUNE)
ds_val = val_raw.map(prep_dataset, tf.data.AUTOTUNE)

## Model development

In [306]:
EMBEDDING_DIM = 128
RNN_UNITS = 32
ATTENTION_UNITS = 16

In [298]:
rnn_kwargs = dict(
    dropout=0.2, 
    recurrent_dropout=0.2, 
    recurrent_initializer="glorot_uniform",
)

In [299]:
class ShapeChecker():
    def __init__(self):
    # Keep a cache of every axis-name seen
        self.shapes = {}

    def __call__(self, tensor, names, broadcast=False):
        if not tf.executing_eagerly():
            return

        parsed = einops.parse_shape(tensor, names)

        for name, new_dim in parsed.items():
            old_dim = self.shapes.get(name, None)

            if (broadcast and new_dim == 1):
                continue

            if old_dim is None:
                # If the axis name is new, add its length to the cache.
                self.shapes[name] = new_dim
                continue

            if new_dim != old_dim:
                raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                                 f"    found: {new_dim}\n"
                                 f"    expected: {old_dim}\n")

In [300]:
class Encoder(M.Model):
    def __init__(self, rnn_units: int, vocab_size: int, embedding_dim: int):
        super(Encoder, self).__init__()
        self.embedding = L.Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)
        self.rnn = L.Bidirectional(
            layer=L.GRU(units=rnn_units, return_sequences=True, **rnn_kwargs),
            merge_mode="sum",
        )
    def call(self, inputs):
        x = self.embedding(inputs)
        outputs = self.rnn(x)
        return outputs

In [347]:
class AdditiveAttention(L.Layer):
    def __init__(self, units: int):
        super().__init__()
        self.w1 = L.Dense(units=units, use_bias=False)
        self.w2 = L.Dense(units=units, use_bias=False)
        self.v = L.Dense(1, use_bias=False)

    def call(self, query, keys):
        shape_checker = ShapeChecker()
        shape_checker(query, "batch t_enc rnn_units")
        shape_checker(keys, "batch t_dec emb_dec")
        # add nexaxis for broadcasting addition along the time dimensions
        scores_query = tf.expand_dims(self.w1(query), axis=2)
        scores_keys = tf.expand_dims(self.w2(keys), axis=1)
        scores = self.v(tf.tanh(tf.add(scores_query, scores_keys)))
        scores = tf.squeeze(scores)
        shape_checker(scores, "batch t_enc t_dec")
        attention_scores = tf.math.softmax(scores, axis=1)  # for each t_enc, unit sum
        context = tf.matmul(attention_scores, keys)
        return context, attention_scores

In [353]:
class Decoder(M.Model):
    def __init__(self, rnn_units: int, vocab_size: int, embedding_dim: int, attention_units: int):
        super(Decoder, self).__init__()
        self.embedding = L.Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)
        self.rnn = L.GRU(units=rnn_units, return_state=True, return_sequences=True, **rnn_kwargs)
        self.attention = AdditiveAttention(units=attention_units)
        self.out = L.Dense(units=vocab_size)
        self.last_attention_scores = None
    def call(self, inputs, enc_hidden_states, state=None, return_state: bool = False):
        shape_checker = ShapeChecker()
        shape_checker(inputs, "batch t_dec")
        shape_checker(enc_hidden_states, "batch t_enc rnn_units")
        #
        emb = self.embedding(inputs)
        shape_checker(emb, "batch t_dec emb")
        #
        rnn_out, rnn_state = self.rnn(emb, initial_state=state)
        shape_checker(rnn_out, "batch t_dec rnn_units")
        #
        context, attention_scores = self.attention(rnn_out, enc_hidden_states)
        self.last_attention_scores = attention_scores
        shape_checker(context, "batch t_dec rnn_units")
        logits = self.out(context)
        shape_checker(logits, 'batch t_dec vocab_size')
        out = (logits, state) if return_state else logits
        return out

In [354]:
shape_checker = ShapeChecker()
enc_inputs, dec_inputs = ds_train.take(1).as_numpy_iterator().next()[0]
shape_checker(enc_inputs, "batch t_enc")

encoder = Encoder(rnn_units=RNN_UNITS, embedding_dim=EMBEDDING_DIM, vocab_size=VOCAB_SIZE)
enc_outputs = encoder(enc_inputs)
shape_checker(enc_outputs, "batch t_enc rnn_units")

decoder = Decoder(rnn_units=RNN_UNITS, embedding_dim=EMBEDDING_DIM, vocab_size=VOCAB_SIZE, attention_units=16)
logits = decoder(dec_inputs[:, :3], enc_outputs)

In [355]:
logits.shape

TensorShape([64, 3, 30000])

In [356]:
class Translator(M.Model):
    def __init__(
        self, 
        rnn_units: int = RNN_UNITS, 
        vocab_size: int = VOCAB_SIZE, 
        embedding_dim: int = EMBEDDING_DIM, 
        attention_units: int = ATTENTION_UNITS,
    ):
        super(Translator, self).__init__()
        self.encoder = Encoder(rnn_units=rnn_units, embedding_dim=embedding_dim, vocab_size=vocab_size)
        self.decoder = Decoder(rnn_units=rnn_units, embedding_dim=embedding_dim, vocab_size=vocab_size, attention_units=attention_units)
    
    def call(self, inputs_enc, inputs_dec):
        enc_hidden_states = self.encoder(inputs_enc)
        logits = self.decoder(inputs_dec, enc_hidden_states)
        return logits

In [357]:
translator = Translator()

In [360]:
translator(enc_inputs, dec_inputs).shape

TensorShape([64, 6, 30000])

In [230]:
enc_inputs.shape

(64, 8)

In [231]:
enc_outputs.shape

TensorShape([64, 8, 32])

In [324]:
dec_inputs[:, :3].shape

(64, 3)

In [326]:
decoder_emb = decoder.embedding(dec_inputs[:, :3])
decoder_emb.shape

TensorShape([64, 3, 128])

In [328]:
dec_rnn_out, state = decoder.rnn(decoder_emb)
print(dec_rnn_out.shape)
print(state.shape)

(64, 3, 32)
(64, 32)


In [329]:
attention = AdditiveAttention(units=16)

In [336]:
scores_keys = tf.expand_dims(attention.w1(enc_outputs), axis=1)
scores_keys.shape

TensorShape([64, 1, 8, 16])

In [337]:
scores_values = tf.expand_dims(attention.w2(dec_rnn_out), axis=2)
scores_values.shape

TensorShape([64, 3, 1, 16])

In [338]:
scores = attention.v(tf.tanh(tf.add(scores_keys, scores_values)))
scores = tf.squeeze(scores)  # drop last dimension
scores.shape

TensorShape([64, 3, 8])

In [339]:
attention_scores = tf.math.softmax(scores, axis=-1)
attention_scores.shape

TensorShape([64, 3, 8])

In [340]:
tf.matmul(attention_scores, enc_outputs).shape

TensorShape([64, 3, 32])